<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/RAG_Gradio_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
api_key = ""

# Cài đặt thư viện cần thiết

In [33]:
!pip install --quiet -U langchain chromadb langchain-openai pypdf gradio

# Xử lí văn bản PDF

In [35]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
loader = PyPDFLoader(r"/content/A_UAV-Based_Aircraft_Surface_Defect_Inspection_System_via_External_Constraints_and_Deep_Learning.pdf")
splits = loader.load_and_split(text_splitter)

# Đưa văn bản thành Embeddings và Lưu trữ vào vectorstore

In [36]:
emb = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = Chroma.from_documents(documents=splits, embedding=emb)

# Xây Dựng Gradio

In [37]:
import gradio as gr
from openai import OpenAI

llm = OpenAI(api_key=api_key)

RAG_PROMPT = """Nhiệm vụ của bạn là trả lời câu hỏi hoặc tóm tắt văn bản của người dùng dựa trên dữ liệu được cho.
Phải trả lời càng dài càng tốt.
Nếu dữ liệu được cho không liên quan đến câu hỏi hoặc yêu cầu, vui lòng trả lời "Tôi không biết"
---
Dữ liệu: {context}
---
Câu hỏi: {question}
---
Trả lời:"""

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    docs = vectorstore.similarity_search(message)
    context = docs[0].page_content
    history_openai_format.append({"role": "user", "content": RAG_PROMPT.format(context=context, question=message)})

    response = llm.chat.completions.create(model='gpt-3.5-turbo',
      messages= history_openai_format,
      temperature=1.0,
      stream=True)


    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message



gr.ChatInterface(predict).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ee510b1cdf7da214bf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
